## Example processing Sentinel-2 data (locally)

In [ ]:
import datacube
import matplotlib.pyplot as plt
from pyproj import Transformer
import time

In [ ]:
# Initialise datacube

dc = datacube.Datacube()

In [ ]:
# (Central NSW)
x_min, x_max = 1200000, 1300000
y_min, y_max = -3600000, -3700000
date_range = ("2024-01-01", "2024-02-28")

In [ ]:
# Load datasets (lazy)

product = "ga_s2bm_ard_3"  # Sentinel-2 B
measurements = ["nbart_red", "nbart_blue", "oa_s2cloudless_mask"]
output_crs = "EPSG:3577"
resolution = [-30, 30]

dask_chunks = {
    "time": 1,
    "y": 500,
    "x": 500
}

ds = dc.load(product=product,
             measurements=measurements,
             crs="EPSG:3577",
             x=(x_min, x_max),
             y=(y_min, y_max),
             time=date_range,
             output_crs=output_crs,
             resolution=resolution,
             dask_chunks=dask_chunks,
             dataset_predicate=lambda ds: ds.metadata.dataset_maturity == "final",
             skip_broken_datasets=True  # Important!
             )

In [ ]:
ds

In [ ]:
# Mask clouds

no_clouds_ds = ds.where(ds["oa_s2cloudless_mask"] == 1)

In [ ]:
no_clouds_ds

In [ ]:
# Visualize individual time slices

TIME_INDEX = 1

q = no_clouds_ds["nbart_red"].isel(time=TIME_INDEX)
q.plot.imshow(cmap="viridis")  # or cmap='gray', 'RdYlGn', etc.
plt.title("Result")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# Example of local processing

ratio_ds = no_clouds_ds["nbart_red"] / no_clouds_ds["nbart_blue"]
mean_ratio_ds = ratio_ds.mean(dim="time", skipna=True)

In [ ]:
mean_ratio_ds

In [ ]:
%%time
mean_ratio_ds = mean_ratio_ds.compute()

In [ ]:
mean_ratio_ds

In [ ]:
# Visualise mean ratio dataset

band = mean_ratio_ds

# Plot with xarray’s wrapper around matplotlib
band.plot.imshow(cmap="viridis")  # or cmap='gray', 'RdYlGn', etc.
plt.title("Result")
plt.xlabel("x")
plt.ylabel("y")
plt.show()